# Análise dos Resultados - Metodologias Qualificação

In [24]:
import os
import sys
from pprint import pprint
import warnings
warnings.filterwarnings(action="ignore")

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from IPython.display import display

from collections import Counter

from tqdm import tqdm, tqdm_notebook
from glob import glob 


# Configurações

In [25]:
# Path do arquivo H5 (base REDD ja preparada p/ NILMTK) e outros insumos fundamentais
caminho_dados = "D:/Projetos/phd-thesis/datasets/"

# Definir diretorios onde iremos salvar os insumos gerados do notebook (dados, imagens, etc.)
caminho_dados_notebook = os.path.join(caminho_dados, "21") # Num. notebook
if not os.path.isdir(caminho_dados_notebook):
    os.makedirs(caminho_dados_notebook)
caminho_imagens_notebook = os.path.join(caminho_dados_notebook, "imagens") # Num. notebook
if not os.path.isdir(caminho_imagens_notebook):
    os.makedirs(caminho_imagens_notebook)

# Path do arquivo H5 (base REDD ja preparada p/ NILMTK)
caminho_redd = os.path.join(caminho_dados, "REDD/low_freq")

# Path completo do arquivo REDD
arquivo_dataset = os.path.join(caminho_redd, "redd.h5")

# VARIAVEL AUXILIAR
# Path dos arquivos relacionados as janelas
caminho_janelas = os.path.join(caminho_redd, "../../phd")
if not os.path.isdir(caminho_janelas):
    os.makedirs(caminho_janelas)

# Análise dos Resultados

In [26]:
# Consolidando planilhas
dados = {}

for dir_residencia in tqdm_notebook(glob(os.path.join(caminho_dados, "20/residencia_*"))):
    id_residencia = dir_residencia.split("_")[-1]
    planilhas = glob(os.path.join(dir_residencia, "resultados_*_cv.xlsx"))
    if len(planilhas) > 0:
        df = pd.concat([pd.read_excel(p) for p in planilhas])
        df["residencia"] = id_residencia
        dados[id_residencia] = df
del df

## Resultados por Residencia

In [38]:
# Metodologia analsada
metodologia = "Baseline"

# Agrupando resultados das varias residencias para a metodologia escolhida
dados_residencias = pd.concat([v[v["metodologia"] == metodologia] for k,v in dados.items()])

In [54]:
# Resultados
df_resultados_residencias = dados_residencias.groupby("residencia").agg({
    "acuracia": ["mean"],
    "f1": ["mean"]
})
df_resultados_residencias

,acuracia,f1
,mean,mean
residencia,,
1,0.959446,0.818075
2,0.946402,0.800484
3,0.965471,0.709121
5,0.952504,0.888746


## Resultados por Aparelho

In [55]:
# Resultados
df_resultados_aparelhos = dados_residencias.groupby(["residencia","aparelho","base"]).agg({
    "acuracia": ["mean"],
    "f1": ["mean"]
})
display(df_resultados_aparelhos)

acuracia        f1
                                  mean      mean
residencia aparelho                             
1          dish_washer - 9    0.915978  0.497023
           fridge - 7         1.000000  1.000000
           microwave - 16     0.901253  0.758513
           washer_dryer - 13  0.988750  0.907097
           washer_dryer - 14  0.991250  0.927742
2          dish_washer - 9    0.967200  0.681547
           fridge - 7         1.000000  1.000000
           microwave - 16     0.784224  0.490871
           washer_dryer - 13  0.990000  0.912424
           washer_dryer - 14  0.990588  0.917576
3          dish_washer - 9    0.965555  0.576105
           fridge - 7         0.983214  0.960020
           microwave - 16     0.939900  0.581492
           washer_dryer - 13  0.967747  0.716226
           washer_dryer - 14  0.967065  0.706890
5          dish_washer - 9    0.958833  0.709275
           fridge - 7         0.906667  0.891833
           microwave - 16     0.840395  0.840163
           washer_dryer - 13  0.990409  0.944987
           washer_dryer - 14  0.990409  0.944987

In [57]:
# Persisindo resultados
xls_writer = pd.ExcelWriter(os.path.join(caminho_dados_notebook, "resultados.xlsx"), engine='xlsxwriter')

df_resultados_residencias.to_excel(xls_writer, sheet_name='Residencias')
df_resultados_aparelhos.to_excel(xls_writer, sheet_name='Aparelhos')

xls_writer.save()